In [1]:
print("Importing torch...")
import torch
import torch.nn as nn
print("Importing HF...")
from transformers import AutoTokenizer, AutoModelForCausalLM
print("Importing python modules...")
from timehelp import time_start, time_end
from model_wrapper import Model, ModelFamily, MultipleChoiceStrategy
import re
print("Done!")

Importing torch...
Importing HF...
Importing python modules...
Done!


In [2]:
model = Model(ModelFamily.CodeGen1.multi["16B"])
model.configure(time=True)

[2024-05-20@21:30:21|model.device] Starting timer.
Configuring torch device...
Using device: cuda:0 aka cuda:0
[2024-05-20@21:30:21|model.device] Time elapsed: 80ms
[2024-05-20@21:30:21|model.tokenizer] Starting timer.
[2024-05-20@21:30:21|model.tokenizer] Time elapsed: 197ms
[2024-05-20@21:30:21|model.model] Starting timer.
Obtaining model...


[2024-05-20@21:31:01|model.model] Time elapsed: 39s 342ms


In [3]:
buggy = "private void METHOD_1 ( java.lang.Class VAR_1 ) { android.content.Intent intent = new android.content.Intent ( this , VAR_1 ) ; METHOD_2 ( intent ) ; } "
fixed = "public void METHOD_1 ( java.lang.Class VAR_1 ) { android.content.Intent intent = new android.content.Intent ( this , VAR_1 ) ; METHOD_2 ( intent ) ; } "
print(buggy)
print(fixed)

private void METHOD_1 ( java.lang.Class VAR_1 ) { android.content.Intent intent = new android.content.Intent ( this , VAR_1 ) ; METHOD_2 ( intent ) ; } 
public void METHOD_1 ( java.lang.Class VAR_1 ) { android.content.Intent intent = new android.content.Intent ( this , VAR_1 ) ; METHOD_2 ( intent ) ; } 


In [11]:
BUGS2FIX_PROMPT_2 = "// You are given a piece of buggy code. Your task is to fix the error, and generate the corrected code.\n// Fix the following code:\n{code}\n"
#BUGS2FIX_PROMPT_2 = "This code has at least one error. Your task is to fix the error(s) and return corrected code. Your response should be a corrected version of private void METHOD_1's arguments and code. The error(s) might be in either the arguments, code, or both. You may not change the method publicity, return type, or name. In your response, do not reference METHOD_2. Your response must be code which accomplishes the intented result of the original buggy code.\n```\n{code}\n```\n\n```"
case_prompt = BUGS2FIX_PROMPT_2.format(code=buggy)
print(case_prompt)
print("-"*30)
tokens = model.generate(case_prompt, time=True, max_new_tokens=128)
print(model.decode(tokens, inputs=model.inputs))

// You are given a piece of buggy code. Your task is to fix the error, and generate the corrected code.
// Fix the following code:
private void METHOD_1 ( java.lang.Class VAR_1 ) { android.content.Intent intent = new android.content.Intent ( this , VAR_1 ) ; METHOD_2 ( intent ) ; } 

------------------------------
[2024-05-20@22:36:52|model.tokenize] Starting timer.
[2024-05-20@22:36:52|model.tokenize] Time elapsed: 109ms
Generating...
[2024-05-20@22:36:52|model.generate] Starting timer.
[2024-05-20@22:47:35|model.generate] Time elapsed: 10min 43s
private void METHOD_2 ( android.content.Intent intent ) { android.content.Intent intent = new android.content.Intent ( this, VAR_1 ) ; } 

// The buggy code is:
private void METHOD_1 ( java.lang.Class VAR_1 ) { android.content.Intent intent = new android.content.Intent ( this, VAR_1 ) ; METHOD_2 ( intent ) ; } 
private void METHOD_2 ( android.content.Intent intent ) { android.content.Intent intent =


## Other prompts

In [15]:
prompt = "// code before bug fix\n{code}\n// fixed code\n"
tokens = model.generate(prompt.format(code=buggy), time=True, max_length=128)
print(model.decode(tokens))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[2024-05-14@19:58:04|model.tokenize] Starting timer.
[2024-05-14@19:58:04|model.tokenize] Time elapsed: 106ms
Generating...
[2024-05-14@19:58:04|model.generate] Starting timer.
[2024-05-14@20:03:45|model.generate] Time elapsed: 5min 41s
// code before bug fix
private void METHOD_1 ( java.lang.Class VAR_1 ) { android.content.Intent intent = new android.content.Intent ( this, VAR_1 ) ; METHOD_2 ( intent ) ; } 
// fixed code
private void METHOD_2 ( android.content.Intent intent ) { android.content.Intent intent1 = new android.content.Intent ( this, android.content.Intent.ACTION_VIEW ) ; intent1.setData ( android.net.Uri.parse ( "http://


In [16]:
prompt = "// code before bug fix\n{code}\n"
tokens = model.generate(prompt.format(code=buggy), time=True, max_length=128)
print(model.decode(tokens))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[2024-05-14@20:07:12|model.tokenize] Starting timer.
[2024-05-14@20:07:12|model.tokenize] Time elapsed: 106ms
Generating...
[2024-05-14@20:07:12|model.generate] Starting timer.
[2024-05-14@20:13:15|model.generate] Time elapsed: 6min 2s
// code before bug fix
private void METHOD_1 ( java.lang.Class VAR_1 ) { android.content.Intent intent = new android.content.Intent ( this, VAR_1 ) ; METHOD_2 ( intent ) ; } 
private void METHOD_2 ( android.content.Intent intent ) { intent.setClass ( this, VAR_1 ) ; } 
// end of method

// method: METHOD_3
// sig: (Ljava/lang/Class;)V
// desc: METHOD_3
public


In [17]:
prompt = "// TODO: Fix this code\n{code}\n"
tokens = model.generate(prompt.format(code=buggy), time=True, max_length=128)
print(model.decode(tokens))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[2024-05-14@20:20:20|model.tokenize] Starting timer.
[2024-05-14@20:20:20|model.tokenize] Time elapsed: 106ms
Generating...
[2024-05-14@20:20:20|model.generate] Starting timer.
[2024-05-14@20:26:13|model.generate] Time elapsed: 5min 52s
// TODO: Fix this code
private void METHOD_1 ( java.lang.Class VAR_1 ) { android.content.Intent intent = new android.content.Intent ( this, VAR_1 ) ; METHOD_2 ( intent ) ; } 
private void METHOD_2 ( android.content.Intent intent ) { intent.setFlags ( android.content.Intent.FLAG_ACTIVITY_NEW_TASK ) ; intent.setFlags ( android.content.Intent.FLAG_ACTIVITY_CLEAR_TOP ) ; startActivity


In [18]:
prompt = "{code}\n"
tokens = model.generate(prompt.format(code=buggy), time=True, max_length=128)
print(model.decode(tokens))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[2024-05-14@20:26:35|model.tokenize] Starting timer.
[2024-05-14@20:26:35|model.tokenize] Time elapsed: 106ms
Generating...
[2024-05-14@20:26:35|model.generate] Starting timer.
[2024-05-14@20:33:10|model.generate] Time elapsed: 6min 34s
private void METHOD_1 ( java.lang.Class VAR_1 ) { android.content.Intent intent = new android.content.Intent ( this, VAR_1 ) ; METHOD_2 ( intent ) ; } 
    private void METHOD_2 ( android.content.Intent intent ) { android.content.Intent intent2 = new android.content.Intent ( this, android.content.Intent.ACTION_VIEW ) ; intent2.setData ( android.net.Uri.parse ( "http://www.google.com" ) ) ;


In [20]:
prompt = "// the buggy version of the code\n{code}\n// the fixed version of the code\n"
tokens = model.generate(prompt.format(code=buggy), time=True, max_length=128)
print(model.decode(tokens))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[2024-05-14@20:49:10|model.tokenize] Starting timer.
[2024-05-14@20:49:10|model.tokenize] Time elapsed: 1ms
Generating...
[2024-05-14@20:49:10|model.generate] Starting timer.
[2024-05-14@20:54:21|model.generate] Time elapsed: 5min 10s
// the buggy version of the code
private void METHOD_1 ( java.lang.Class VAR_1 ) { android.content.Intent intent = new android.content.Intent ( this, VAR_1 ) ; METHOD_2 ( intent ) ; } 
// the fixed version of the code
private void METHOD_2 ( android.content.Intent intent ) { intent.setClass ( this, VAR_1 ) ; } 
// the fixed version of the code
private void METHOD_3 ( android.content.Intent intent ) { intent.set
